## 모듈 선언

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

`from soynlp.normalizer import *`  
한국어 분석을 위한 pure python code
- 학습데이터를 이용하지 않으면서 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법을 지향합니다.
- [Github Link](https://github.com/lovit/soynlp)

---



In [ ]:
import gc

# 메모리 해제
gc.collect()

## 파일 읽기

In [ ]:
train_data_path ="./train_ai_last.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data

In [ ]:
# train_data_path ="/aiffel/aiffel/dktc/data/train.csv"
# train_data = pd.read_csv(train_data_path)

---

## 데이터 확인

## 문장 전처리 함수 선언

### 1) `preprocess_sentence()`
1. 영어, 한국어가 아닌 경우 공백 (` `) 처리
2. 두 개 이상의 느낌표(`!+`)가 있을 경우 느낌표 하나로 처리
3. 두 개 이상의 물음표(`\?+`)가 있을 경우 물음표 하나로 처리
4. `?`, `.`, `!`, `,` 가 있을 경우 그 주위에 공백을 추가
5. 연속적인 공백이 있을 시 공백을 하나로 처리
6. 문장 앞뒤의 공백과 개행문자를 제거(`strip`)

In [ ]:
def preprocess_sentence(sentence):
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence

### 2) `preprocess_sentence2()`
1. 영어, 한국어가 아닌 경우 공백 (` `) 처리
2. 연속적인 공백이 있을 시 공백을 하나로 처리
3. 문장 앞뒤의 공백과 개행문자를 제거(`strip`)
4. 위 함수를 거친 문장은 문법기호(`?`, `!`, `,` 등)도 모두 제거됨

In [ ]:
def preprocess_sentence2(sentence):
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣])', " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence

---

## 문장 전처리 함수를 사용하여 학습할 문장(`sentences`) 설정

In [ ]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation'], desc="Generate sentences.."):
    # preprocess_sentence2()로 문장(val)을 전처리하여 배열에 저장
    sentences.append(preprocess_sentence2(val))

In [ ]:
#### preprocess_sentence() 케이스 사용 시 해당 함수 내용을 써보자
#### 테스트는 preprocess_sentence()를 거친 데이터 기준으로 수행됨
def dummy():
    # 학습할 문장이 담길 배열
    sentences2 = []

    for val in tqdm(train_data['conversation'], desc="Generate sentences.."):
        # preprocess_sentence()로 문장(val)을 전처리하여 배열에 저장
        sentences.append(preprocess_sentence(val))
#####

---

## 대화 종류(`class`) 문장을 숫자로 변환

---

`협박 관련 대화` &rarr; `0`  
`갈취 관련 대화` &rarr; `1`  
`직장 관련 대화` &rarr; `2`  
`기타 관련 대화` &rarr; `3`  

In [ ]:
labels = []

for val in tqdm(train_data['class'], desc="class label convert to num..."):    
    if '갈취' in val:
        labels.append(1)
    if '기타' in val:
        labels.append(3)
    if '직장' in val:
        labels.append(2)
    if '협박' in val:
        labels.append(0)

---

## 데이터셋(`sentences`, `labels`)을 8:2 (`train`:`test`)로 분할

In [ ]:
# 데이터셋 분할
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

In [ ]:
def random_deletion(words, p=0.3):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return ''.join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_swap(words, n=3):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

In [ ]:
print('증강 전 : ', len(train_sentences))

In [ ]:
train_splted = pd.DataFrame({
    'sentence': train_sentences,
    'class':train_labels
})
train_splted_rd = train_splted.copy()
train_splted_rd['sentence'] = train_splted_rd['sentence'].apply(random_deletion)



train_splted_rs = train_splted.copy()
train_splted_rs['sentence'] = random_swap(train_splted_rs['sentence'])


In [ ]:
train_concated = pd.concat([train_splted,train_splted_rd,train_splted_rs])
train_concated

In [ ]:
# train_sentences = list(train_concated['sentence'].values)
# train_labels = list(train_concated['class'].values)

---

## BERT 토크나이저 & 모델 준비
학습된 `BERT` 모델 사용 &rarr; `bert-base-multilingual-cased`

In [ ]:
# BERT 토크나이저와 모델 준비
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=4)

---

## 모델 파라미터 선언

In [ ]:
# 토큰 최대 길이
MAX_LEN = 200
# 데이터 묶음 크기
BATCH_SIZE = 16
# Learning Rate
lr = 5e-5 # 5e-5 , 1e-4 
# 훈련 횟수
EPOCH = 3

---

## 일반 문장을 BERT 입력 형식으로 변환
`예시` &rarr; `[CLS] 안녕하세요 [SEP]`

In [ ]:
# 데이터셋을 BERT 입력 형식으로 변환
train_encodings = tokenizer(train_sentences, truncation=True, padding=True, max_length=MAX_LEN) # 뒤쪽에 패딩
val_encodings = tokenizer(val_sentences, truncation=True, padding=True, max_length=MAX_LEN)

---

## TF 데이터셋 생성

In [ ]:
# TensorFlow 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(100).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(BATCH_SIZE)


---

## 모델 컴파일

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

---

---

## 모델 훈련

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',    # 모니터링할 지표 (검증 정확도)
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

In [ ]:
epoch = EPOCH
model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=epoch,
    callbacks=[reduce_lr],
)

In [ ]:
# lr 변경 1e-5 -> 5e-5 


In [ ]:
import gc

# 메모리 해제
gc.collect()

---

## 모델 평가

In [ ]:
# 모델 평가
evaluation = model.evaluate(val_dataset)
print("평가 결과:", evaluation)

---

## 테스트 문장으로 모델 평가하기

### test.json 읽기

In [ ]:
test_data_path ="/aiffel/aiffel/dktc/data/test.json"

with open(test_data_path, "r", encoding="utf-8") as json_file:
    test = json.load(json_file)

---

### JSON의 key별로 가지는 문장(`value`) 읽으면서 예측해보기

In [ ]:
import numpy as np

test_predicst = list()

for key in test:
    test_sentence = test[key]['text']
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding=True, max_length=128, return_tensors="tf")
    
    test_predictions = model.predict({
        "input_ids": test_encodings["input_ids"],
        "token_type_ids": test_encodings["token_type_ids"],
        "attention_mask": test_encodings["attention_mask"]
    }) # [ 0.7805823,  2.6188664, -2.0281641, -0.9672525]
    test_class_probabilities = tf.nn.softmax(test_predictions.logits, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predicst.append(test_predicted_class[0])

---

# submission
`submission.csv`파일을 위한 DataFrame 생성

In [ ]:
def labelnum_to_text(x):
    if x == 1 : # 갈취
        return '01'
    if x == 2 : # 직장
        return '02'
    if x == 3 : # 기타
        return '03'
    if x == 0 : # 협박 
        return '00'

submission = pd.DataFrame({'class':test_predicst}, index=list(test.keys()))
submission['class'] = submission['class'].apply(labelnum_to_text)
submission

---

## 날짜별 파일 만들기

In [ ]:
import datetime

# 현재 날짜와 시간 가져오기
now = datetime.datetime.now()

# 날짜와 시간을 원하는 형식으로 포맷팅
date_time_str = now.strftime("%Y%m%dT%H%M")

# 파일명 생성
file_name = f"submission{date_time_str}.csv"

# submission.csv 파일을 날짜패턴 합쳐 만들기
submission.to_csv(file_name)

### `preprocess_sentence2()` 전처리 출발 케이스